In [1]:
import json
import pandas as pd

In [2]:
%matplotlib inline

In [3]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 999)

In [5]:
%%time
from pymongo import MongoClient

MONGO_HOST= 'mongodb://localhost/twitterdata'
client = MongoClient(MONGO_HOST)
db = client.twitterdata
coll = db.abortolegal

tweets = []
users = {}
cols_tweet = ['id', 'created_at', 'favorite_count', 'retweet_count', 'full_text',
              'posicion', 'favorited', 'retweeted']
cols_user = [
             'screen_name',
             'location',
             'followers_count',
             'statuses_count',
             'description',
             'friends_count',
             'favourites_count',
             'created_at',
             'id',
             'name',             
             'verified']

for t in coll.find():
    tr = [t[c] for c in cols_tweet] + [t['user']['name'], t['user']['id']]
    tr.append(t['retweeted_status']['id'] if t['retweeted'] else t['id'])
    tweets.append(tr)
    uid = t['user']['id'] 
    if uid not in users:
        users[uid] = [t['user'][c] for c in cols_user]

CPU times: user 4min 45s, sys: 7.33 s, total: 4min 52s
Wall time: 4min 53s


In [6]:
%%time
tweets = pd.DataFrame(tweets, columns=cols_tweet + ['user_name', 'user_id', 'original_id'])

CPU times: user 2.27 s, sys: 60.1 ms, total: 2.33 s
Wall time: 2.34 s


In [7]:
%%time
users = pd.DataFrame(users.values(), columns=cols_user)

CPU times: user 967 ms, sys: 11.9 ms, total: 978 ms
Wall time: 977 ms


## Guardar datos en formato accesible

In [8]:
tweets.head()

,id,created_at,favorite_count,retweet_count,full_text,posicion,favorited,retweeted,user_name,user_id,original_id
0,1005600427080110080,Sat Jun 09 23:59:59 +0000 2018,0,14,"RT @PTSarg: Contamos con una ventaja que ellos no tienen: nuestra decisión, nuestra convicción y nuestra fuerza de lucha. #NoVotenContraLas…",si,False,False,Luciana HL,3130302891,1005600427080110080
1,1005600422772527105,Sat Jun 09 23:59:58 +0000 2018,0,59,"RT @juanabalmedina: Excelentes reflexiones de una médica ginecóloga, católica practicante, que dice “Me repugna un país donde después de un…",si,False,False,Oriana Ribeiro,383350997,1005600422772527105
2,1005600422269243397,Sat Jun 09 23:59:58 +0000 2018,2,0,En el fondo lo que les jode es nuestra libertad #NoVotenContraLasMujeres #AbortoLegalYA,si,False,False,Luz Scarpati,2999619141,1005600422269243397
3,1005600416862801921,Sat Jun 09 23:59:57 +0000 2018,0,0,"holla! me gustaría q ayuden a esta junta d firmas para el aborto seguro y gratuito, gracias \n\nAprueben la Ley de Aborto Seguro y Gratuito @DiputadosAR #abortolegalya #13JAbortoLegal https://t.co/aT9bKgrh9P via @ChangeorgAR",si,False,False,gigi not hadid,3668114236,1005600416862801921
4,1005600413838692352,Sat Jun 09 23:59:56 +0000 2018,0,308,"RT @femialborto: A vos que decís #Salvemoslasdosvidas te pido que escuches esto, que intentes abrir la mente y entender las palabras, son m…",si,False,False,niña de tilcara🌰,828222093804371968,1005600413838692352


In [14]:
clean_text = lambda s: s.replace('\n', ' ').replace('\r', ' ')

In [15]:
tweets.full_text = tweets.full_text.apply(clean_text)

In [16]:
tweets.to_csv('../data/tweets.csv', index=False, encoding='utf8')

In [17]:
users.description = users.description.apply(clean_text)

In [18]:
users.head()

,screen_name,location,followers_count,statuses_count,description,friends_count,favourites_count,created_at,id,name,verified
0,beautyinarose,"Escalada casi Banfield,Bs As",427,50613,"Taladrista por adopción, pero ya en la sangre.Intento de comunicologa/guionista, probable quedar en intento. Terca idealista de nacimiento.",791,10199,Fri Jan 23 15:38:24 +0000 2009,19398656,Jules Caccaglio,False
1,DanielMort_,"Celaya, Guanajuato",722,18823,~ Admin de @DeditosHelp & Mod de @dedreviil ~ LVL 16 ~,178,13596,Fri Oct 28 03:17:25 +0000 2016,791841268879851520,DanielMort_ 🗡,False
2,VioleGallotti98,"Córdoba, Argentina",1390,6576,Bariloche ‘15.,1107,13425,Fri Jul 25 18:07:57 +0000 2014,2680160258,νισlєтα gαllσттι,False
3,Lucia_Achigar,Montevideo,631,3130,Fan de las versalitas.,876,11690,Fri Mar 04 00:15:24 +0000 2011,260492029,Lucía Achigar 💚,False
4,florgimenezc,"Córdoba, Argentina",1174,25947,Veintitrés del cuatro del noventa y seis. Derecho en la Universidad Nacional de Córdoba,360,9978,Tue May 08 16:59:21 +0000 2012,574707030,peto,False


In [19]:
users.to_csv('../data/users.csv', index=False, encoding='utf8')